# Question 6: Quantum Phase Kickback

## Problem Statement
How does the phase kickback mechanism work, and why is it fundamental to quantum algorithms?

## Background

**Phase kickback** is a crucial quantum computing technique where:
- A controlled operation applies a phase to the control qubit
- Instead of affecting the target qubit's basis states
- The phase "kicks back" to the control qubit

This mechanism is essential for:
- Deutsch-Jozsa algorithm
- Grover's search algorithm  
- Quantum phase estimation
- Shor's factoring algorithm

## Key Concept

When a controlled-U gate operates on an eigenstate |ψ⟩ of U:
```
CU |a⟩|ψ⟩ = e^(iφa) |a⟩|ψ⟩
```
The eigenvalue e^(iφ) "kicks back" as a phase on the control qubit!

In [ ]:
# Import required libraries
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info import Statevector
import numpy as np
import matplotlib.pyplot as plt

## Simple Example: Controlled-Z Gate

The Z gate has eigenstates |0⟩ and |1⟩ with eigenvalues +1 and -1:
- Z|0⟩ = +1·|0⟩ (eigenvalue = +1, phase = 0)
- Z|1⟩ = -1·|1⟩ (eigenvalue = -1, phase = π)

Let's see phase kickback in action!

In [ ]:
# Circuit demonstrating phase kickback with CZ gate
qc = QuantumCircuit(2)

# Put control qubit in superposition
qc.h(0)

# Put target qubit in |1⟩ (eigenstate of Z with eigenvalue -1)
qc.x(1)

# Apply controlled-Z
# When control is |1⟩, applies Z to target
# Since target is eigenstate |1⟩, phase -1 kicks back to control
qc.cz(0, 1)

# Apply H to see the phase in the computational basis
qc.h(0)

# Draw circuit
print("Phase Kickback Circuit:")
qc.draw(output='mpl')

In [ ]:
# Get the statevector
simulator = Aer.get_backend('statevector_simulator')
result = execute(qc, backend=simulator).result()
statevector = result.get_statevector()

print("Final statevector:")
print(statevector)

# Measure to see effect
qc_measure = qc.copy()
qc_measure.measure_all()

qasm_sim = Aer.get_backend('qasm_simulator')
counts = execute(qc_measure, backend=qasm_sim, shots=1000).result().get_counts()
print("\nMeasurement results:")
plot_histogram(counts, title='Phase Kickback Effect')

## Explanation

### Without phase kickback (target in |0⟩):
Initial: |+⟩|0⟩ = (|0⟩ + |1⟩)/√2 ⊗ |0⟩

After CZ: (|0⟩ + |1⟩)/√2 ⊗ |0⟩ (no change, Z|0⟩ = |0⟩)

After final H: |0⟩|0⟩ (always measure 0 on first qubit)

### With phase kickback (target in |1⟩):
Initial: |+⟩|1⟩ = (|0⟩ + |1⟩)/√2 ⊗ |1⟩

After CZ: (|0⟩ - |1⟩)/√2 ⊗ |1⟩ (phase kicks back! This is |-⟩|1⟩)

After final H: |1⟩|1⟩ (always measure 1 on first qubit)

**The phase kickback flipped our measurement result from 0 to 1!**

## Application: Deutsch Algorithm

The Deutsch algorithm uses phase kickback to determine if a function is constant or balanced in a single query.

Given a black-box function f: {0,1} → {0,1}, determine if:
- **Constant**: f(0) = f(1)
- **Balanced**: f(0) ≠ f(1)

Classical: Requires 2 queries

Quantum: Requires 1 query (using phase kickback!)

In [ ]:
def deutsch_oracle(qc, oracle_type='balanced'):
    """
    Create an oracle for Deutsch algorithm
    oracle_type: 'constant_0', 'constant_1', 'balanced_identity', 'balanced_not'
    """
    if oracle_type == 'constant_0':
        # f(x) = 0 for all x
        pass  # Do nothing
    elif oracle_type == 'constant_1':
        # f(x) = 1 for all x
        qc.x(1)
    elif oracle_type == 'balanced_identity':
        # f(x) = x
        qc.cx(0, 1)
    elif oracle_type == 'balanced_not':
        # f(x) = NOT x
        qc.x(0)
        qc.cx(0, 1)
        qc.x(0)
    return qc

def deutsch_algorithm(oracle_type='balanced_identity'):
    """Execute Deutsch algorithm"""
    qc = QuantumCircuit(2, 1)
    
    # Initialize: |0⟩|1⟩
    qc.x(1)
    
    # Create superposition: |+⟩|-⟩
    qc.h(0)
    qc.h(1)
    
    qc.barrier()
    
    # Apply oracle (phase kickback happens here!)
    deutsch_oracle(qc, oracle_type)
    
    qc.barrier()
    
    # Apply H to extract phase information
    qc.h(0)
    
    # Measure
    qc.measure(0, 0)
    
    return qc

# Test with balanced oracle
qc_balanced = deutsch_algorithm('balanced_identity')
print("Deutsch Algorithm (Balanced Oracle):")
qc_balanced.draw(output='mpl')

In [ ]:
# Run for all oracle types
oracle_types = ['constant_0', 'constant_1', 'balanced_identity', 'balanced_not']
results = {}

simulator = Aer.get_backend('qasm_simulator')

for oracle in oracle_types:
    qc = deutsch_algorithm(oracle)
    counts = execute(qc, backend=simulator, shots=1000).result().get_counts()
    results[oracle] = counts
    
    # Interpret result
    if '0' in counts and counts['0'] > 900:
        function_type = "CONSTANT"
    else:
        function_type = "BALANCED"
    
    print(f"{oracle:20s}: {counts} → {function_type}")

## How Deutsch Algorithm Uses Phase Kickback

1. **Setup**: Control in |+⟩, target in |-⟩
2. **Oracle**: Applies controlled-f operation
3. **Phase Kickback**: 
   - If f is constant: no phase change
   - If f is balanced: π phase change
4. **Final H**: Converts phase to measurement outcome
   - Constant → measure |0⟩
   - Balanced → measure |1⟩

**One query solves the problem!**

## Key Insight

Phase kickback allows us to:
1. Store information in quantum phases (not just basis states)
2. Interfere these phases constructively/destructively
3. Extract the answer through measurement

This is the foundation of quantum computational advantage!

## Answer & Key Takeaways

### What is Phase Kickback?
When a controlled-unitary acts on an eigenstate, the eigenvalue phase "kicks back" to the control qubit.

### Why It Matters:
1. **Quantum Parallelism**: Evaluate function on superposition of inputs
2. **Phase Storage**: Store information in relative phases
3. **Quantum Advantage**: Enable algorithms impossible classically

### Used In:
- Deutsch-Jozsa algorithm
- Grover's search (oracle marks solutions with phase)
- Quantum phase estimation (foundation of Shor's algorithm)
- Variational quantum algorithms

### The Magic:
Phase kickback transforms the question "What is f(x)?" into "Does f flip the phase?" - a question quantum computers can answer in superposition!